In [3]:
# ============================
# RSS / Web → news_th.csv
# ============================
# ใช้ได้บน Colab/เครื่องส่วนตัว
# สร้างไฟล์ news_th.csv: date,symbol,text,source,url

%pip -q install feedparser beautifulsoup4 pandas python-dateutil dateparser lxml html5lib requests

import feedparser, requests, hashlib, re, time, os
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timezone, timedelta
import dateparser

# ----------------------------
# 1) ตั้งค่าแหล่งข่าว (เพิ่ม/ลดได้)
# ----------------------------
# แนะนำใช้ Google News RSS (ยืดหยุ่น, อัปเดตไว) + RSS สำนักข่าวที่มี feed
FEEDS = [
    # Google News (Thai)
    {"url": "https://news.google.com/rss/search?q=SET+หุ้นไทย+เมื่อวานนี้&hl=th&gl=TH&ceid=TH:th", "symbol": "^SET", "source": "GoogleNews SET"},
    {"url": "https://news.google.com/rss/search?q=ตลาดหุ้นไทย+OR+หุ้นไทย+เมื่อวานนี้&hl=th&gl=TH&ceid=TH:th", "symbol": "^SET", "source": "GoogleNews TH Stocks"},
    # ถ้าอยากติดตามหุ้นรายตัว ให้เพิ่ม keyword:
    {"url": "https://news.google.com/rss/search?q=PTT.BK+หุ้น&hl=th&gl=TH&ceid=TH:th", "symbol": "SET.BK", "source": "GoogleNews SET"},
    # ตัวอย่าง RSS สำนักข่าว (ใส่เฉพาะที่คุณใช้จริง)
    {"url": "https://www.kaohoon.com/rss", "symbol": "^SET", "source": "ข่าวหุ้นออนไลน์"},
    {"url": "https://thunhoon.com/rss", "symbol": "^SET50", "source": "Nation Business"}
]

# ----------------------------
# 2) ตัวเลือกการดึงเนื้อข่าวจากหน้าเว็บ (optional)
# ----------------------------
SCRAPE_FULL_TEXT = False  # True = ลองเข้าไปอ่านจากลิงก์ (ช้า/อาจเจอ paywall)
HTTP_TIMEOUT = 10
REQUEST_HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; FinanceAI/1.0)"}
MAX_PER_FEED = 200         # จำกัดจำนวนต่อ feed เพื่อกันยาวเกิน

# เก็บไฟล์ออก
OUT_CSV = "news_th.csv"

# ----------------------------
# 3) Utilities
# ----------------------------
def clean_html(html):
    # ลบแท็ก HTML -> ข้อความล้วน
    soup = BeautifulSoup(html or "", "lxml")
    # ลบ script/style
    for bad in soup(["script","style"]):
        bad.decompose()
    text = soup.get_text(" ", strip=True)
    # ลดช่องว่างต่อเนื่อง
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def fetch_full_text(url):
    # ดึงเนื้อข่าวจากหน้าเว็บแบบง่าย (ไม่ robust เท่าข่าวบางเว็บ)
    try:
        r = requests.get(url, timeout=HTTP_TIMEOUT, headers=REQUEST_HEADERS)
        if r.status_code != 200:
            return ""
        soup = BeautifulSoup(r.text, "lxml")
        # heuristic: รวม <p> ทั้งหมด
        ps = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
        txt = " ".join(ps)
        txt = re.sub(r"\s+", " ", txt).strip()
        return txt[:5000]  # กันยาวเกิน
    except Exception:
        return ""

def normalize_date(entry):
    # รับ datetimeจาก feedparser หรือแปลงจาก string -> datetime (UTC)
    dt = None
    if "published" in entry and entry.published:
        dt = dateparser.parse(entry.published)
    elif "updated" in entry and entry.updated:
        dt = dateparser.parse(entry.updated)
    elif "published_parsed" in entry and entry.published_parsed:
        dt = datetime(*entry.published_parsed[:6])
    elif "updated_parsed" in entry and entry.updated_parsed:
        dt = datetime(*entry.updated_parsed[:6])
    if not dt:
        dt = datetime.utcnow()
    if not dt.tzinfo:
        dt = dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def hash_key(*parts):
    m = hashlib.sha256()
    for p in parts:
        m.update(str(p or "").encode("utf-8"))
    return m.hexdigest()

# ----------------------------
# 4) ดึงข่าวจากทุก feed
# ----------------------------
rows = []
seen = set()

for feed in FEEDS:
    url = feed["url"]
    symbol = feed.get("symbol", "^SET50")
    source = feed.get("source", url)

    print(f"Fetching RSS: {source} | {url}")
    d = feedparser.parse(url)
    count = 0

    for e in d.entries[:MAX_PER_FEED]:
        title = clean_html(getattr(e, "title", "") or "")
        summary = clean_html(getattr(e, "summary", "") or "")
        link = getattr(e, "link", "") or ""
        date_utc = normalize_date(e)
        # รวม title + summary (ถ้าเปิดขูดเว็บ จะพยายามแทนด้วยเนื้อข่าวเต็ม)
        text = (title + ". " + summary).strip()

        if SCRAPE_FULL_TEXT and link:
            full = fetch_full_text(link)
            if len(full) > len(text) * 1.3:  # ถ้า full ยาวกว่าเยอะ -> น่าจะดีกว่า summary
                text = full

        # กันข่าวซ้ำด้วย hash ของ (link หรือ text)
        key = hash_key(link or text[:200])
        if key in seen:
            continue
        seen.add(key)

        # เก็บผลลัพธ์
        rows.append({
            "date": date_utc.date().isoformat(),  # เก็บเป็น YYYY-MM-DD
            "symbol": symbol,
            "text": text[:5000],                  # ตัดความยาวกันยาวเกิน
            "source": source,
            "url": link
        })
        count += 1
    print(f"  -> fetched {count} items")

print(f"\nTotal collected: {len(rows)} items from {len(FEEDS)} feeds")

# ----------------------------
# 5) สร้าง DataFrame + เซฟ CSV
# ----------------------------
if len(rows) == 0:
    print("⚠️ ไม่พบข่าวจาก FEEDS ที่กำหนด ลองเพิ่ม/เปลี่ยน feed หรือเปิด SCRAPE_FULL_TEXT")
else:
    df_news = pd.DataFrame(rows).sort_values("date")
    # ลบแถวว่าง/ข้อความสั้นมาก
    df_news = df_news[df_news["text"].str.len() > 15].copy()
    df_news.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ Saved {OUT_CSV} | shape={df_news.shape}")
    display(df_news.head(10))


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 12.0 MB/s eta 0:00:00
Fetching RSS: GoogleNews SET | https://news.google.com/rss/search?q=SET+หุ้นไทย+เมื่อวานนี้&hl=th&gl=TH&ceid=TH:th
  -> fetched 100 items
Fetching RSS: GoogleNews TH Stocks | https://news.google.com/rss/search?q=ตลาดหุ้นไทย+OR+หุ้นไทย+เมื่อวานนี้&hl=th&gl=TH&ceid=TH:th
  -> fetched 85 items
Fetching RSS: GoogleNews SET | https://news.google.com/rss/search?q=PTT.BK+หุ้น&hl=th&gl=TH&ceid=TH:th
  -> fetched 15 items
Fetching RSS: ข่าวหุ้นออนไลน์ | https://www.kaohoon.com/rss
  -> fetched 10 items
Fetching RSS: Nation Business | https://thunhoon.com/rss
  -> fetched 0 items

Total collected: 210 items from 5 feeds
✅ Saved news_th.csv | shape=(210, 5)


,date,symbol,text,source,url
193,2015-06-09,SET.BK,"น้ำตาจะไหล !! แอพหุ้นบน iPhone, Apple Watch, M...",GoogleNews SET,https://news.google.com/rss/articles/CBMivwFBV...
199,2020-05-22,SET.BK,"""Power"" Sector (22 พ.ค.63) - bangkokbiznews. ""...",GoogleNews SET,https://news.google.com/rss/articles/CBMiWkFVX...
196,2020-09-16,SET.BK,10 หุ้น (SET100) อัตรากำไรขั้นต้นสูงปรี๊ดดด! -...,GoogleNews SET,https://news.google.com/rss/articles/CBMiYEFVX...
191,2021-03-22,SET.BK,15 หุ้นบิ๊กเบิ้มแกร่งสุด ใน 5 เซกเตอร์จอมพลัง ...,GoogleNews SET,https://news.google.com/rss/articles/CBMiYEFVX...
190,2023-02-27,SET.BK,หุ้นไทยวันนี้(27 ก.พ.66) ลบ 6.67 จุด ขาย PTT-I...,GoogleNews SET,https://news.google.com/rss/articles/CBMiSkFVX...
197,2024-01-18,SET.BK,วิเคราะห์หุ้นรายตัว : บล.เคจีไอฯ PTTGC ประมาณก...,GoogleNews SET,https://news.google.com/rss/articles/CBMiZkFVX...
198,2024-06-18,SET.BK,วิเคราะห์หุ้น : บล.เคจีไอฯ ENERGY SECTOR OPEC+...,GoogleNews SET,https://news.google.com/rss/articles/CBMiZkFVX...
195,2024-08-05,SET.BK,วิเคราะห์หุ้นรายตัว : บล.เคจีไอฯ PTT กำไรใน 2Q...,GoogleNews SET,https://news.google.com/rss/articles/CBMiZkFVX...
189,2025-01-20,SET.BK,วิเคราะห์หุ้นรายตัว : บล.เคจีไอฯ PTTGC ฟ้ามักม...,GoogleNews SET,https://news.google.com/rss/articles/CBMiZkFVX...
99,2025-02-06,^SET,ประเด็นร้อน: หุ้นไทย Big Cap มีปัจจัยลบเฉพาะตั...,GoogleNews SET,https://news.google.com/rss/articles/CBMitAFBV...
